In [ ]:
# Setup imports and path
from pathlib import Path
import pandas as pd
import json
from polymarket_analysis.data.markets import load_raw_markets
from polymarket_analysis.data.load_history import load_price_history
from polymarket_analysis.data.binance import Binance

pd.set_option('display.max_colwidth', None)  

In [2]:
%matplotlib tk


In [3]:
raw_markets = load_raw_markets()

print(f"Loaded {len(raw_markets)} markets.")

Loaded 45616 markets.


In [7]:
filtered_markets = raw_markets[
    raw_markets['question'].str.contains('Bitcoin|BTC|crypto', case=False, na=False, regex=True) 
    # raw_markets['question'].str.contains('June 24', case=False, na=False, regex=True)
]

tokens_with_slugs = [
    {
        "slug": row['market_slug'],
        "outcome": token['outcome'],
        "token_id": token['token_id']
    }
    for _, row in filtered_markets.iterrows()
    for token in row['tokens'] if token['outcome'] != ''  and token['token_id'] != ''
]

In [21]:
try:
    token_histories = [
        load_price_history(token_id=market['token_id'], slug=market['slug'], outcome=market['outcome'])
        for market in tokens_with_slugs if market['slug'] == 'bitcoin-above-103000-on-june-20'
    ]
except Exception as e:
    print(f"Error loading price history: {e}")

In [22]:
token_histories

[PriceHistory(slug='bitcoin-above-103000-on-june-20', outcome='Yes', token_id='23369195288648180659357313269757613104295332489620235641529131236791515394564', price_df=              timestamp  price
 0   2025-06-13 15:00:08  0.575
 1   2025-06-13 15:10:08  0.750
 2   2025-06-13 15:20:08  0.665
 3   2025-06-13 15:30:08  0.675
 4   2025-06-13 15:40:07  0.745
 ..                  ...    ...
 894 2025-06-19 20:00:08  0.715
 895 2025-06-19 20:10:07  0.700
 896 2025-06-19 20:20:07  0.725
 897 2025-06-19 20:30:07  0.710
 898 2025-06-19 20:38:07  0.715
 
 [899 rows x 2 columns]),
 PriceHistory(slug='bitcoin-above-103000-on-june-20', outcome='No', token_id='28690244775797077493520457350695870577692994709973478957963085765560773779976', price_df=              timestamp  price
 0   2025-06-13 15:00:08  0.425
 1   2025-06-13 15:10:08  0.250
 2   2025-06-13 15:20:08  0.335
 3   2025-06-13 15:30:08  0.325
 4   2025-06-13 15:40:07  0.255
 ..                  ...    ...
 894 2025-06-19 20:00:08  0.285

In [49]:
# Convert first row to JSON
import json

# Get the first row as a dictionary
first_row = filtered_markets.iloc[0].to_dict()

# Convert to JSON with pretty formatting
first_row_json = json.dumps(first_row, indent=2, default=str)

print("First row as JSON:")
print(first_row_json)

First row as JSON:
{
  "enable_order_book": true,
  "active": true,
  "closed": false,
  "archived": false,
  "accepting_orders": true,
  "accepting_order_timestamp": "2025-06-17T17:23:45Z",
  "minimum_order_size": 5,
  "minimum_tick_size": 0.01,
  "condition_id": "0x15d51789d6f1c77c4c588ecd4b317d33595ba4e1022e8b9dc536e7e0c86ef8fb",
  "question_id": "0xc314fe6a8fa235982f3969cd6e4b72b2c2df116c0a626a9af992c79af619d7e4",
  "question": "Bitcoin above $103,000 on June 24?",
  "description": "This market will resolve to \"Yes\" if the Binance 1 minute candle for BTC/USDT 12:00 in the ET timezone (noon) on the date specified in the title has a final \u201cClose\u201d price higher than the price specified in the title. Otherwise, this market will resolve to \"No\".\n\nThe resolution source for this market is Binance, specifically the BTC/USDT \"Close\" prices currently available at https://www.binance.com/en/trade/BTC_USDT with \u201c1m\u201d and \u201cCandles\u201d selected on the top bar.\n\

In [50]:
print(filtered_markets[['question', 'end_date_iso']].to_string())

                                                               question          end_date_iso
45518                                Bitcoin above $103,000 on June 24?  2025-06-24T00:00:00Z
45523  Will the price of Bitcoin be between $102K and $104K on June 24?  2025-06-24T00:00:00Z
45533  Will the price of Bitcoin be between $104K and $106K on June 24?  2025-06-24T00:00:00Z
45554          Will the price of Bitcoin be less than $100K on June 24?  2025-06-24T00:00:00Z
45575       Will the price of Bitcoin be greater than $106K on June 24?  2025-06-24T00:00:00Z
45596  Will the price of Bitcoin be between $100K and $102K on June 24?  2025-06-24T00:00:00Z


In [8]:
polymarket_start = token_histories[0].price_df['timestamp'].min()
polymarket_end = token_histories[0].price_df['timestamp'].max()

btc_df = Binance.load_bitcon_5min(
    from_date=polymarket_start, 
    to_date=polymarket_end
)
print(f"Filtered Bitcoin data: {len(btc_df)} rows")

Filtered Bitcoin data: 325 rows


In [27]:
# %matplotlib tk
%matplotlib inline

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()
# Create subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

for mh in token_histories:
    fig.add_trace(go.Scatter(
            x=mh.price_df['timestamp'], 
            y=mh.price_df['price'],
            mode='lines+markers',
            name=f"{mh.slug}",
        ))

# Add Bitcoin price
fig.add_trace(
    go.Scatter(
        x=btc_df['timestamp'], 
        y=btc_df['price'],
        mode='lines',
        name='Bitcoin Price',
        line=dict(color='black'),
    ),
    secondary_y=True,
)

# Update layout
fig.update_xaxes(title_text="Timestamp")
fig.update_yaxes(title_text="Polymarket Price", secondary_y=False)
fig.update_yaxes(title_text="Bitcoin Price (USD)", secondary_y=True)

fig.update_layout(
    title_text="Polymarket vs Bitcoin Price Comparison (5-minute data)",
    legend=dict(
        orientation="h",  # horizontal orientation
        yanchor="top",
        y=-0.1,          # position below the plot
        xanchor="center",
        x=0.5
    ),
    showlegend=True
)

fig.show(renderer="browser")